In [52]:
import glob
import os
import time

import cv2
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from torch.utils.tensorboard import SummaryWriter

from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#### 数据预处理


In [53]:
path = '../flower_photos/'
w = 100
h = 100
c = 3

In [54]:
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)] 
    imgs = []
    labels = []
    for idx, folder in enumerate(cate):
        for im in glob.glob(folder + '/*.jpg'):
            img = cv2.imread(im)
            img = cv2.resize(img, (w, h))
            imgs.append(img)
            labels.append(idx)
    return np.asarray(imgs, np.float32), np.asarray(labels, np.int32)

In [55]:
data, label = read_img(path)

data = torch.FloatTensor(data).permute(0, 3, 1, 2)
label = torch.LongTensor(label)

data.shape, label.shape

(torch.Size([739, 3, 100, 100]), torch.Size([739]))

In [56]:
seed = 109
np.random.seed(seed)

(x_train, x_val, y_train, y_val) = train_test_split(data, label, test_size=0.20, random_state=seed)
x_train = x_train / 255
x_val = x_val / 255

flower_dict = {0:'bee', 1:'blackberry', 2:'blanket', 3:'bougainvilliea', 4:'bromelia', 5:'foxglove'}
class_list = ['bee', 'blackberry', 'blanket', 'bougainvilliea', 'bromelia', 'foxglove']

#### TORCH框架

In [57]:
class FlowerDataset(Dataset):
    def __init__(self, data, label):
        super(FlowerDataset, self).__init__()
        self.data = data
        self.label = label
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

In [58]:
'''
    model
    采取和tf一样的CNN架构
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv_model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(12*12*64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 6),
        )

    def forward(self, data):
        x = self.conv_model(data)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

In [59]:
num_epochs = 100

#### TRAIN AND EVAL

In [60]:
class Stat:
    def __init__(self, training, writer=None):
        self.step = 0
        self.loss = []
        self.labels = []
        self.pred_labels = []
        self.training = training
        self.writer = writer
    
    def add(self, pred, labels, loss):
        labels = labels.numpy()
        pred = pred.detach().numpy()
        pred_labels = np.argmax(pred, axis = 1)
        self.loss.append(loss)
        self.labels.extend(labels)
        self.pred_labels.extend(pred_labels)

    def log(self):
        self.step += 1
        acc = accuracy_score(self.labels, self.pred_labels)
        loss = sum(self.loss) / len(self.loss)
        self.loss = []
        self.labels = []
        self.pred_labels = []
        if not self.writer:
            return loss, acc
        if self.training:
            self.writer.add_scalar('train_loss', loss, self.step)
            self.writer.add_scalar('train_acc', acc, self.step)
        else:
            self.writer.add_scalar('dev_loss', loss, self.step)
            self.writer.add_scalar('dev_acc', acc, self.step)
        return loss, acc

In [61]:
def train(model, train_data_loader, dev_data_loader):
    loss_func = CrossEntropyLoss()
    optimizer = Adam(
        model.parameters(),
        lr=0.001,
        weight_decay=0.01,
    )

    writer = SummaryWriter('./summary/' + time.strftime('%m-%d_%H.%M', time.localtime()))
    train_stat = Stat(training=True, writer=writer)
    dev_stat = Stat(training=False, writer=writer)


    best_acc, best_net = 0.0, None
    for epoch in range(num_epochs):
        print(f"--- epoch: {epoch + 1} ---")
        for iter, batch in enumerate(train_data_loader):
            model.train()
            data, labels = batch[0], batch[1]
            pred_outputs = model(data)
            loss = loss_func(pred_outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_stat.add(pred_outputs, labels, loss.item())
            
        train_loss, train_acc = train_stat.log()

        model.eval()
        with torch.no_grad():
            for batch in dev_data_loader:
                data, labels = batch[0], batch[1]
                pred_outputs = model(data)
                loss = loss_func(pred_outputs, labels)
                dev_stat.add(pred_outputs, labels, loss.item())
        dev_loss, dev_acc = dev_stat.log()
        print(  f"training loss: {train_loss:.4f}, acc: {train_acc:.2%}, " \
                f"dev loss: {dev_loss:.4f}, acc: {dev_acc:.2%}.")

        if dev_acc > best_acc:
            best_acc = dev_acc
            best_net = deepcopy(model.state_dict())
            
    print(f"best dev acc: {best_acc:.4f}")
    return best_net

In [62]:
model_path = f'C:\\Users\zzzgry\Desktop\midspore_lab2\model\\pytorch\\model-'+time.strftime('%m-%d_%H.%M', time.localtime())+'.pkl'

In [63]:
train_set = FlowerDataset(x_train, y_train)
dev_set = FlowerDataset(x_val, y_val)

train_data_loader = DataLoader(train_set, 32, True)
dev_data_loader = DataLoader(dev_set, 32, False)

In [64]:
net = MyModel()
best_net = train(net, train_data_loader, dev_data_loader)

--- epoch: 1 ---
training loss: 1.6555, acc: 29.61%, dev loss: 1.4652, acc: 40.54%.
--- epoch: 2 ---
training loss: 1.4485, acc: 42.64%, dev loss: 1.3301, acc: 46.62%.
--- epoch: 3 ---
training loss: 1.2410, acc: 50.25%, dev loss: 1.1420, acc: 56.76%.
--- epoch: 4 ---
training loss: 1.1846, acc: 54.82%, dev loss: 1.0716, acc: 64.86%.
--- epoch: 5 ---
training loss: 1.0426, acc: 63.45%, dev loss: 1.0192, acc: 62.16%.
--- epoch: 6 ---
training loss: 0.9621, acc: 66.33%, dev loss: 1.0660, acc: 62.16%.
--- epoch: 7 ---
training loss: 0.9658, acc: 63.96%, dev loss: 1.0012, acc: 64.19%.
--- epoch: 8 ---
training loss: 0.9580, acc: 64.47%, dev loss: 1.0590, acc: 56.76%.
--- epoch: 9 ---
training loss: 0.9199, acc: 66.16%, dev loss: 1.0345, acc: 61.49%.
--- epoch: 10 ---
training loss: 0.9128, acc: 66.50%, dev loss: 0.9766, acc: 63.51%.
--- epoch: 11 ---
training loss: 0.8681, acc: 66.67%, dev loss: 0.9111, acc: 66.22%.
--- epoch: 12 ---
training loss: 0.8240, acc: 71.24%, dev loss: 0.8728, ac

In [65]:
torch.save(best_net, model_path)
net.load_state_dict(torch.load(model_path))

<All keys matched successfully>

#### TEST

In [66]:
def test(model, test_data_loader):
    test_stat = Stat(training=False)
    model.eval()
    with torch.no_grad():
        for batch in test_data_loader:
            data, labels = batch[0], batch[1]
            pred_outputs = model(data)
            test_stat.add(pred_outputs, labels, 0)

    report = classification_report(
        test_stat.labels,
        test_stat.pred_labels,
        target_names=class_list,
        digits=4,
        zero_division=0,
    )
    print(report)

In [67]:
path_test = '../TestImages/'
imgs=[]
for im in glob.glob(path_test+'/*.jpg'):
    img=cv2.imread(im)           
    img=cv2.resize(img,(w,h)) 
    imgs.append(img)  
imgs = np.asarray(imgs,np.float32)    

test_data = imgs / 255
test_label = [x for x in range(len(test_data))]

test_data = torch.FloatTensor(test_data).permute(0, 3, 1, 2)
test_label = torch.LongTensor(test_label)
test_data.shape, test_label.shape

(torch.Size([6, 3, 100, 100]), torch.Size([6]))

In [68]:
test_set = FlowerDataset(test_data, test_label)
test_data_loader = DataLoader(test_set, batch_size=len(test_data), shuffle=False)
test(net, test_data_loader)

                precision    recall  f1-score   support

           bee     1.0000    1.0000    1.0000         1
    blackberry     1.0000    1.0000    1.0000         1
       blanket     1.0000    1.0000    1.0000         1
bougainvilliea     1.0000    1.0000    1.0000         1
      bromelia     1.0000    1.0000    1.0000         1
      foxglove     1.0000    1.0000    1.0000         1

      accuracy                         1.0000         6
     macro avg     1.0000    1.0000    1.0000         6
  weighted avg     1.0000    1.0000    1.0000         6



In [69]:
net = MyModel()
net.load_state_dict(torch.load(model_path))
test(net, test_data_loader)

                precision    recall  f1-score   support

           bee     1.0000    1.0000    1.0000         1
    blackberry     1.0000    1.0000    1.0000         1
       blanket     1.0000    1.0000    1.0000         1
bougainvilliea     1.0000    1.0000    1.0000         1
      bromelia     1.0000    1.0000    1.0000         1
      foxglove     1.0000    1.0000    1.0000         1

      accuracy                         1.0000         6
     macro avg     1.0000    1.0000    1.0000         6
  weighted avg     1.0000    1.0000    1.0000         6

